# 데이터의 수집 방법
1. 크롤링
   - 자동화된 스크립트 사용하여 웹 페이지를 탐색하고 데이터 수집
   - 전체 사이트 또는 다수의 페이지를 탐색
   - 검색 엔진의 크롤러와 유사
2. 스크래핑
   - 특정 웹 페이지에서 필요한 데이터 추출
   - 웹 크롤링의 하위 집합으로서 특정 데이터만 수집
  
* 웹 크롤링 -> 조직적, 자동화 된 방법으로 월드 와이드 웹을 탐색하는 컴퓨터 프로그램
* 크롤러 -> 웹 상의 정보를 수집하고 체계화하는 역할을 담당

# 데이터 수집(스크래핑)
- requests
  - HTTP 요청을 보내고 응답을 받기 위한 라이브러리
- BeautifulSoup
  - HTML 및 XML 문서를 파싱하여 원하는 데이터를 추출하기 위한 라이브러리
- Scrapy
  - 크롤링 및 스크래핑을 위한 프레임워크
  - 정적 웹 크롤링 강점
- Selenium
  - 웹 브라우저 자동화를 위한 라이브러리
  - 동적 웹 스크래핑 강점

In [1]:
import requests
from bs4 import BeautifulSoup

# AI HUB 페이지 URL
url = 'https://www.aihub.or.kr/'

#웹 페이지 요청
response = requests.get(url)
response.raise_for_status() # 요청이 성공했는지 확인

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.content, 'html.parser')

# 인기 데이터 TOP3 섹션 찾기
top3_section = soup.find('div', class_='secR')

# 각 데이터 항목 추출
data_list = top3_section.find_all('div', class_='list')

# 각 데이터 항목 추출
titles = []
for data in data_list:
    title = data.find('h3').get_text(strip = True)
    clean_title = title.split(']')[-1].strip()
    titles.append(clean_title)
    
# 추출한 데이터 출력
for idx, title in enumerate(titles, start = 1):
    print(f"TOP {idx}: {title}")




TOP 1: 객체 간 관계성 인지용 한국형 비전 데이터
TOP 2: 손∙팔 협조에 의한 파지-조작 동작 데이터
TOP 3: 상용 자율주행차 야간 자동차 전용도로 데이터


2. requests 라이브러리 및 BeautifulSoup 사용 예제

In [2]:
import requests
from bs4 import BeautifulSoup

# 1. 웹 페이지 요청
url = 'https://ko.wikipedia.org/wiki/위키백과:대문'
response = requests.get(url)

# 2. 요청이 성공했는지 확인
if response.status_code == 200:
    # 3. BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #4. 페이지 제목 추출
    title = soup.find('h1', id = 'firstHeading').text
    print(f"Title: {title}")
    
    #5. 첫 번째 단락 추출
    first_paragraph = soup.find('p').text
    print(f"First paragraph: {first_paragraph}")
else:
    print(f"failed to retrieve the web page. Status code: {response.status_code}")

Title: 위키백과:대문
First paragraph:  위키백과



3. Selenium 라이브러리 사용 예제

In [3]:
!pip install selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By

#URL 성정
URL = "https://ko.wikipedia.org/wiki/위키백과:대문"

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless") #브라우저 창을 띄우지 않음
options.add_argument('--disavle-dev-shm-usage')
options.add_argument("--no-snadbox")

# 웹 드라이버 설정
driver = webdriver.Chrome(options=options)

try:
    # 위키백과 대문 페이지 열기
    driver.get(URL)
    
    #"우리 모두가 만들어가는 자유 백과사전"과 "문서 이하 내용 추출"
    main_content = driver.find_element(By.CSS_SELECTOR, "#mw-content-text > div.mw-content-ltr.mw-parser-output > div.main-box.main-top > div > p:nth-child(2)").text
    print("Main Content:", main_content)
finally:
    # 웹 드라이브 종료
    driver.quit()

Main Content: 우리 모두가 만들어가는 자유 백과사전
문서 675,058개와 최근 기여자 1,766명


4. Scrapy 라이브러리 사용 예제

In [1]:
!pip install scrapy

In [2]:
!scrapy startproject wikipedia_scraper

New Scrapy project 'wikipedia_scraper', using template directory '/opt/anaconda3/lib/python3.10/site-packages/scrapy/templates/project', created in:
    /Users/chhy/Desktop/KakaoTech/Practical_exercise/Day1/wikipedia_scraper

You can start your first spider with:
    cd wikipedia_scraper
    scrapy genspider example example.com


In [6]:
import scrapy

class WikipediaSpider(scrapy.Spider):
    name = "wikipedia"
    start_urls = [
        'https://ko.wikipedia.org/wiki/위키백과:대문',
    ]

    def parse(self, response):
        main_content = response.css('#mw-content-text > div.mw-content-ltr.mw-parser-output > div.main-pane > div.main-pane-right > div.wikipedia-ko.main-recommended.main-box').get()
        yield {
            'main_content': main_content,
        }

In [9]:
#%%writefile -a wikipedia_scraper/wikipedia_scraper/settings.py

#ROBOTSTXT_OBEY = False

Appending to wikipedia_scraper/wikipedia_scraper/settings.py


In [7]:
# @title
# run_scrapy.py
from scrapy.crawler import CrawlerProcess
from wikipedia_scraper.wikipedia_scraper.spiders.wikipedia_spider import WikipediaSpider

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'FEEDS': {
        'output.json': {
            'format': 'json',
            'encoding': 'utf8',
            'store_empty': False,
            'fields': None,
        
            'indent': 4,
        },
    },
})

process.crawl(WikipediaSpider)
process.start()

2024-07-08 13:36:12 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-07-08 13:36:12 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.12 (main, Jul  5 2023, 15:02:25) [Clang 14.0.6 ], pyOpenSSL 24.0.0 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.5, Platform macOS-14.4-arm64-arm-64bit
2024-07-08 13:36:12 [scrapy.addons] INFO: Enabled addons:
[]
2024-07-08 13:36:12 [py.warnings] WARNING: /opt/anaconda3/lib/python3.10/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_

In [5]:
!python /Users/chhy/Desktop/KakaoTech/Lecture/Lecture_code/Day1/wikipedia_scraper/run_scrapy.py

Traceback (most recent call last):
  File "/Users/chhy/Desktop/KakaoTech/Lecture/Lecture_code/Day1/wikipedia_scraper/run_scrapy.py", line 4, in <module>
    from wikipedia_scraper.wikipedia_scraper.spiders.wikipedia_spider import WikipediaSpider
ModuleNotFoundError: No module named 'wikipedia_scraper.wikipedia_scraper'


In [8]:
# @title
# 출력물을 예쁘게 출력
import json
from pprint import pprint

# 크롤링 결과 파일 로드 및 출력
with open('output.json', 'r') as f:
    data = json.load(f)
    pprint(data)

[{'main_content': '<div class="wikipedia-ko main-recommended main-box" '
                  'style="width: 100%; margin-top: 20px; height: max-content; '
                  'flex: 1;">\n'
                  '\t\t\t<div class="wikipedia-ko good-header" style="width: '
                  '100%; overflow: auto; font-size: 1.1em;"><span '
                  'style="display: inline-block; height: 2rem; line-height: '
                  '2rem; padding-left: .5rem; padding-right: 1rem; margin: '
                  '.6rem 0; border-radius: 0 1rem 1rem 0; background-color: '
                  '#EBEBEB;"><b><a '
                  'href="/wiki/%ED%95%B4%EB%A6%AC%EC%97%87_%ED%84%B0%EB%B8%8C%EB%A8%BC" '
                  'title="해리엇 터브먼">알찬 글: 해리엇 터브먼</a></b></span><div '
                  'style="float: right; height: 2rem; margin: .6rem 0;"><span '
                  'class="plainlinks clickbutton"><a '
                  'href="/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%95%8C%EC%B0%AC_%EA%B8%80" '
  